In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [12]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [13]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [14]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=False, random_state=0)
kf.get_n_splits(X)

5

In [15]:
param_distributions={
    'estimator__max_depth': [5, 8, 15],
    'estimator__min_samples_split': [2,5,10],
    'estimator__min_samples_leaf': [1,5,10],
}

In [16]:
estimator=RandomForestRegressor(n_estimators=500, n_jobs=2)
selector = RFECV(estimator, step=5, cv=5, n_jobs=1)

In [15]:
#estimator.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features=None, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=5,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [16]:
#y_pred=estimator.predict(X_test)

In [17]:
clf= RandomizedSearchCV(selector, param_distributions, scoring="r2", cv=kf, n_jobs=1, random_state=0)

In [18]:
clf.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=False),
          error_score='raise',
          estimator=RFECV(cv=5,
   estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
   n_jobs=1, scoring=None, step=5, verbose=0),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'estimator__max_depth': [5, 8, 15], 'estimator__min_samples_split': [2, 5, 10], 'estimator__min_samples_leaf': [1, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='r2', verbose=0)

In [23]:
clf.best_estimator_

RFECV(cv=5,
   estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
   n_jobs=1, scoring=None, step=5, verbose=0)

In [20]:
y_pred = clf.predict(X_test)

In [21]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('forest-best.csv', index=False)